**Od strzała wam to nie zadziała, trzeba sabie ogarnąć token i plik .env. To jak sobie to ogarniać jest w tym poradniku co wysłałem na teams**

In [79]:
from dotenv import load_dotenv
import os
import base64
from requests import post
import requests
import numpy as np
from sklearn.preprocessing import StandardScaler


load_dotenv()
client_id = os.getenv("CLIENT_ID") 
client_secret = os.getenv("CLIENT_SECRET")

def get_token(client_id, client_secret):
    auth_string = client_id + ":" + client_secret
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")
    token_url = "https://accounts.spotify.com/api/token"

    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }

    data = {
        "grant_type": "client_credentials"
    }

    r = requests.post(token_url, headers=headers, data=data)
    token = r.json()["access_token"]
    return token

def get_auth_header(token):
    return {"Authorization": "Bearer " + token}

def search_for_artist(token, artist_name):
    search_url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)
    query = f"artist:{artist_name}"
    query_url = f"{search_url}?q={query}&type=artist"
    r = requests.get(query_url, headers=headers)
    return r.json()["artists"]["items"][0]["id"]

def get_artist_id(token, artist_name):
    search_url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)
    query = f"artist:{artist_name}"
    query_url = f"{search_url}?q={query}&type=artist"
    r = requests.get(query_url, headers=headers)
    return r.json()["artists"]["items"][0]["id"]

def get_genres(token, artist_id):
    artist_url = f"https://api.spotify.com/v1/artists/{artist_id}"
    headers = get_auth_header(token)
    r = requests.get(artist_url, headers=headers)
    return r.json()["genres"]

def get_recommendations(token, artist_id):
    recommendations_url = "https://api.spotify.com/v1/recommendations"
    headers = get_auth_header(token)
    query = f"seed_artists={artist_id}"
    query_url = f"{recommendations_url}?{query}"
    r = requests.get(query_url, headers=headers)
    return r.json()

def get_tracks(token, artist_id):
    tracks_url = "https://api.spotify.com/v1/artists/{artist_id}/top-tracks"
    headers = get_auth_header(token)
    query = f"country=US"
    query_url = f"{tracks_url}?{query}"
    r = requests.get(query_url, headers=headers)
    return r.json()

def get_albums(token, artist_id):
    albums_url = "https://api.spotify.com/v1/artists/{artist_id}/albums"
    headers = get_auth_header(token)
    query = f"country=US"
    query_url = f"{albums_url}?{query}"
    r = requests.get(query_url, headers=headers)
    return r.json()

def get_audio_features(token, track_id):
    audio_features_url = "https://api.spotify.com/v1/audio-features/{track_id}"
    headers = get_auth_header(token)
    query_url = audio_features_url.format(track_id=track_id)
    r = requests.get(query_url, headers=headers)
    return r.json()

def get_track_id(token, track_name, artist_name):
    search_url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)
    query = f"{track_name} {artist_name}"
    query_url = f"{search_url}?q={query}&type=track"
    r = requests.get(query_url, headers=headers)
    return r.json()["tracks"]["items"][0]["id"]


token = get_token(client_id, client_secret)


In [80]:
search_for_artist(token, "Linkin Park")

'6XyY86QOPPrYVGvF9ch6wz'

In [81]:
get_artist_id(token, "Linkin Park")

'6XyY86QOPPrYVGvF9ch6wz'

In [82]:
get_genres(token, get_artist_id(token, "Linkin Park")) 

['alternative metal', 'nu metal', 'post-grunge', 'rap metal', 'rock']

In [83]:
get_recommendations(token, get_artist_id(token, "Linkin Park"))

{'tracks': [{'album': {'album_type': 'ALBUM',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1caBfBEapzw8z2Qz9q0OaQ'},
      'href': 'https://api.spotify.com/v1/artists/1caBfBEapzw8z2Qz9q0OaQ',
      'id': '1caBfBEapzw8z2Qz9q0OaQ',
      'name': 'Asking Alexandria',
      'type': 'artist',
      'uri': 'spotify:artist:1caBfBEapzw8z2Qz9q0OaQ'}],
    'available_markets': ['AR',
     'AU',
     'AT',
     'BE',
     'BO',
     'BR',
     'BG',
     'CA',
     'CL',
     'CO',
     'CR',
     'CY',
     'CZ',
     'DK',
     'DO',
     'DE',
     'EC',
     'EE',
     'SV',
     'FI',
     'FR',
     'GR',
     'GT',
     'HN',
     'HK',
     'HU',
     'IS',
     'IE',
     'IT',
     'LV',
     'LT',
     'LU',
     'MY',
     'MT',
     'MX',
     'NL',
     'NZ',
     'NI',
     'NO',
     'PA',
     'PY',
     'PE',
     'PH',
     'PL',
     'PT',
     'SG',
     'SK',
     'ES',
     'SE',
     'CH',
     'TW',
     'TR',
     'UY',
     'US',
     '

In [84]:
get_tracks(token, get_artist_id(token, "Linkin Park"))

{'error': {'status': 400, 'message': 'invalid id'}}

In [85]:
get_albums(token, get_artist_id(token, "Linkin Park"))

{'error': {'status': 400, 'message': 'invalid id'}}

In [86]:
get_track_id(token, "Numb", "Linkin Park")

'2nLtzopw4rPReszdYBJU6h'

In [87]:
get_audio_features(token, get_track_id(token, "Numb", "Linkin Park"))

{'danceability': 0.488,
 'energy': 0.87,
 'key': 6,
 'loudness': -4.185,
 'mode': 0,
 'speechiness': 0.0421,
 'acousticness': 0.00353,
 'instrumentalness': 0,
 'liveness': 0.484,
 'valence': 0.211,
 'tempo': 110.072,
 'type': 'audio_features',
 'id': '2nLtzopw4rPReszdYBJU6h',
 'uri': 'spotify:track:2nLtzopw4rPReszdYBJU6h',
 'track_href': 'https://api.spotify.com/v1/tracks/2nLtzopw4rPReszdYBJU6h',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2nLtzopw4rPReszdYBJU6h',
 'duration_ms': 187520,
 'time_signature': 4}

In [88]:
get_audio_features(token, get_track_id(token, "Jak zapomnieć", "Jeden Osiem L"))

{'danceability': 0.722,
 'energy': 0.593,
 'key': 9,
 'loudness': -6.388,
 'mode': 1,
 'speechiness': 0.236,
 'acousticness': 0.48,
 'instrumentalness': 6.71e-06,
 'liveness': 0.147,
 'valence': 0.393,
 'tempo': 84.93,
 'type': 'audio_features',
 'id': '45pmh0kZvulXSgg4uJGB1X',
 'uri': 'spotify:track:45pmh0kZvulXSgg4uJGB1X',
 'track_href': 'https://api.spotify.com/v1/tracks/45pmh0kZvulXSgg4uJGB1X',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/45pmh0kZvulXSgg4uJGB1X',
 'duration_ms': 209040,
 'time_signature': 4}

In [89]:
import requests

def print_track_info(track_id, token):
    headers = {"Authorization": f"Bearer {token}"}

    track_url = f"https://api.spotify.com/v1/tracks/{track_id}"
    audio_features_url = f"https://api.spotify.com/v1/audio-features/{track_id}"

    track_response = requests.get(track_url, headers=headers)
    audio_features_response = requests.get(audio_features_url, headers=headers)

    if track_response.status_code != 200 or audio_features_response.status_code != 200:
        print("Błąd w zapytaniu do API Spotify")
        return None

    track_info = track_response.json()
    audio_features_info = audio_features_response.json()

    info = {
        "duration_ms": track_info["duration_ms"],
        "explicit": track_info["explicit"],
        "danceability": audio_features_info["danceability"],
        "energy": audio_features_info["energy"],
        "key": audio_features_info["key"],
        "loudness": audio_features_info["loudness"],
        "mode": audio_features_info["mode"],
        "speechiness": audio_features_info["speechiness"],
        "acousticness": audio_features_info["acousticness"],
        "instrumentalness": audio_features_info["instrumentalness"],
        "liveness": audio_features_info["liveness"],
        "valence": audio_features_info["valence"],
        "tempo": audio_features_info["tempo"],
        "time_signature": audio_features_info["time_signature"]
    }

    return info


In [90]:
print_track_info(get_track_id(token, "Numb", "Linkin Park"), token)

{'duration_ms': 187520,
 'explicit': False,
 'danceability': 0.488,
 'energy': 0.87,
 'key': 6,
 'loudness': -4.185,
 'mode': 0,
 'speechiness': 0.0421,
 'acousticness': 0.00353,
 'instrumentalness': 0,
 'liveness': 0.484,
 'valence': 0.211,
 'tempo': 110.072,
 'time_signature': 4}

****==================================================================================****

In [91]:
import requests
import pandas as pd

def get_track_info(track_id, token):
    headers = {"Authorization": f"Bearer {token}"}

    track_url = f"https://api.spotify.com/v1/tracks/{track_id}"
    audio_features_url = f"https://api.spotify.com/v1/audio-features/{track_id}"

    track_response = requests.get(track_url, headers=headers)
    audio_features_response = requests.get(audio_features_url, headers=headers)

    if track_response.status_code != 200 or audio_features_response.status_code != 200:
        print("Błąd w zapytaniu do API Spotify")
        return None

    track_info = track_response.json()
    audio_features_info = audio_features_response.json()

    #explicit = bool(track_info["explicit"])

    info = pd.DataFrame({
        "duration_ms": [track_info["duration_ms"]],
        "explicit": [track_info["explicit"]],
        "danceability": [audio_features_info["danceability"]],
        "energy": [audio_features_info["energy"]],
        "key": [audio_features_info["key"]],
        "loudness": [audio_features_info["loudness"]],
        "mode": [audio_features_info["mode"]],
        "speechiness": [audio_features_info["speechiness"]],
        "acousticness": [audio_features_info["acousticness"]],
        "instrumentalness": [audio_features_info["instrumentalness"]],
        "liveness": [audio_features_info["liveness"]],
        "valence": [audio_features_info["valence"]],
        "tempo": [audio_features_info["tempo"]],
        "time_signature": [audio_features_info["time_signature"]]
    })

    return info


In [92]:
track = get_track_info(get_track_id(token, "Numb", "Linkin Park"), token)
track

,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,187520,False,0.488,0.87,6,-4.185,0,0.0421,0.00353,0,0.484,0.211,110.072,4


In [93]:
track.values.shape

(1, 14)

In [94]:
import tensorflow as tf

tensor = tf.convert_to_tensor(track.values, dtype=tf.float32)
tensor

<tf.Tensor: shape=(1, 14), dtype=float32, numpy=
array([[ 1.87520e+05,  0.00000e+00,  4.88000e-01,  8.70000e-01,
         6.00000e+00, -4.18500e+00,  0.00000e+00,  4.21000e-02,
         3.53000e-03,  0.00000e+00,  4.84000e-01,  2.11000e-01,
         1.10072e+02,  4.00000e+00]], dtype=float32)>

In [95]:
import pickle

with open("model.pkl", "rb") as f:
    model = pickle.load(f)

prediction = model.predict(tensor)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


In [96]:
prediction

array([[1., 0., 0., 0., 0.]], dtype=float32)

In [97]:
np.argmax(prediction)

0

In [98]:
def prediction_to_text(prediction):
    if prediction == 0:
        return "Potencjalna popularność utworu bardzo niska"
    elif prediction == 1:
        return "Potencjalna popularność utworu dosyc niska"
    elif prediction == 2:
        return "Potencjalna popularność utworu przecietna"
    elif prediction == 3:
        return "Potencjalna popularność utworu wysoka"
    else:
        return "Utwór przejawia potencjał ogromnego hitu!"

In [99]:
prediction_to_text(np.argmax(prediction)) # tu jest bardzo niska bo dane nie są przeskalowane (a model był trenowany na skalowanych danych)

'Potencjalna popularność utworu bardzo niska'

In [100]:
def predict_new_track(track_id, token, model):
    track = get_track_info(track_id, token)
    scaler = StandardScaler()
    track = scaler.fit_transform(track.values)
    tensor = tf.convert_to_tensor(track, dtype=tf.float32)
    prediction = model.predict(tensor)
    return prediction_to_text(np.argmax(prediction))

In [101]:
predict_new_track(get_track_id(token, "One Dance", "Drake"), token, model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step


'Potencjalna popularność utworu przecietna'

In [102]:
predict_new_track(get_track_id(token, "Numb", "Linkin Park"), token, model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step


'Potencjalna popularność utworu przecietna'

In [103]:
predict_new_track(get_track_id(token, "Despacito", "Luis Fonsi; Daddy Yankee"), token, model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


'Potencjalna popularność utworu przecietna'

Mniej więcej tak wygląda przewidywanie popularności. Tak jak pisałem w tym drugim pliku model raczej słabo sobie radzi i raczej będzie dużo kawałków skazywał na niską popularność ale mamy coś co realnie daje jakieś wyniki.

(**EDIT** Przeskalowałem dane i nagle daje wszystko na srednia szansę na popularność i raczej to nie jest dobrze ehh)

Wrzucenie tego na Kafke będzie wymagało prawdopodobnie zrobienia jakiś klas itd ale wszystkie funkcje które są potrzebne żeby coś wyciągnąć z API i przepuścić przez model i cośtam dostać co wygląda w miare legit